# Projeto do Nanodegree - 2025/1

## Introdução

Este notebook faz parte do projeto do Nanodegree 2025/1 da disciplina de Machine Learning & Inteligência Artificial.
Seu objetivo é analisar, explorar, preparar os dados e aplicar a modelos para a tarefa de previsão de evasão de estudantes em um curso online síncrono promovido pela PensComp.

---

In [ ]:
pip install unidecode

## Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode

## Carregamento dos Dados

O dataset original é carregado a partir de um arquivo CSV contendo informações dos usuários. Este arquivo servirá como base para a análise exploratória e limpeza inicial dos dados.

In [ ]:
df = pd.read_csv('dados_projeto_evasao_treino - Copia.csv')

## Análise Exploratória

In [ ]:
# Estrutura do DF
df.shape
df.info()

In [ ]:
# Amostra dos dados
df.head()

In [ ]:
# Classificação das colunas por tipo de prefixo
categoricas = [col for col in df.columns if col.startswith("ds_")]
temporais = [col for col in df.columns if col.startswith("ts_")]
numericas_nr = [col for col in df.columns if col.startswith("nr_")]
numericas_vl = [col for col in df.columns if col.startswith("vl_")]

print("\nColunas categóricas:", categoricas)
print("Colunas temporais:", temporais)
print("Colunas numéricas (contagem - nr_):", numericas_nr)
print("Colunas numéricas (valores/escalares - vl_):", numericas_vl)

### Verificação de itens ausentes

In [ ]:
# Verificando colunas que possuam ao menos um item nulo
missing = (df.isnull().mean() * 100).sort_values(ascending=True)

print("\nProporção de valores ausentes por coluna (%):")
print(missing[missing > 0].round(2))

## Limpeza e Preparação dos Dados

Primeiro vamos remover a coluna que representaria o ID para não influenciar na predição e após isso remover todas as linhas onde na coluna ts_primeiro_acesso esteja zerado "0", pois isso simboliza que o aluno nunca acessou o portal

In [ ]:
df = df.drop("Unnamed: 0", axis=1)


In [ ]:
df = df[df['ts_primeiro_acesso'] != 0]

In [ ]:
df.head()

Agora fizemos a conversao dos dados TS para um datetime

In [ ]:
df['ts_primeiro_acesso'] = pd.to_datetime(df['ts_primeiro_acesso'], unit='s')
df['ts_ultimo_acesso'] = pd.to_datetime(df['ts_ultimo_acesso'], unit='s')

In [ ]:
df.head()

limpeza e formatação coluna cidade usuario

In [ ]:
print(df['ds_cidade_usuario'].unique())

In [ ]:
df['ds_cidade_usuario'] = (
    df['ds_cidade_usuario']
    .astype(str)
    .str.strip()
    .str.upper()
    .apply(unidecode.unidecode)
)

In [ ]:
df['ds_cidade_usuario'].replace("NAN", "CIDADE NAO INFORMADA", inplace=True)

In [ ]:
print(df['ds_cidade_usuario'].unique())

### Tratamento de Valores Ausentes

Durante a análise, identificamos 5 colunas com altos índices de valores nulos:

- `vl_media_submissoes_codigo` (~99%)
- `vl_submissoes_por_dias_ativos` (~98%)
- `vl_desempenho_questionario` (~93%)
- `vl_engajamento_usuario_por_intervalo` (~92.75%)
- `vl_engajamento_usuario_intradia` (~92.75%)

#### Colunas removidas
As duas primeiras foram removidas por conterem dados ausentes em quase todos os registros, o que inviabiliza sua utilização estatística ou em modelos preditivos.

#### Colunas preenchidas com zero
As demais colunas foram mantidas e os valores nulos preenchidos com **zero**, assumindo que a ausência dos dados representa **falta de atividade do usuário** (ex: nenhum questionário feito, nenhum engajamento detectado). Isso mantém a coerência da análise com o objetivo de detectar evasão por inatividade.


In [ ]:
# Remover colunas com mais de 70% de valores ausentes
df.drop(columns=["vl_media_submissoes_codigo", "vl_submissoes_por_dias_ativos"], inplace=True)
df.drop(columns=["vl_media_questoes_por_dia", "vl_engajamento_notas"], inplace=True)

In [ ]:
# Preencher a coluna de desempenho com zero
df["vl_desempenho_questionario"] = df["vl_desempenho_questionario"].fillna(0)
df["vl_engajamento_usuario_por_intervalo"] = df["vl_engajamento_usuario_por_intervalo"].fillna(0)
df["vl_engajamento_usuario_intradia"] = df["vl_engajamento_usuario_intradia"].fillna(0)
df["vl_desempenho_usuario"] = df["vl_desempenho_usuario"].fillna(0)
df["vl_media_notas"] = df["vl_media_notas"].fillna(0)

### Por que utilizamos a mediana para preencher valores ausentes?

Ao lidar com variáveis contínuas como tempo médio de questionário (`vl_medio_tempo_questionario`) e tempo médio em questionários avaliados (`vl_medio_tempo_questionario_avaliado`), optamos por preencher os valores ausentes com a **mediana**.

A **mediana** é o valor central de uma distribuição ordenada — ou seja, separa os 50% menores dos 50% maiores valores. Diferente da **média**, a mediana **não é afetada por outliers ou valores extremos**, o que a torna mais robusta para representar o "comportamento típico" dos dados, especialmente em distribuições assimétricas ou com grande variação.

#### Vantagens de usar a mediana:
- Evita distorção causada por tempos muito longos ou muito curtos.
- Mantém a coerência estatística da variável.
- Preserva o padrão geral da distribuição sem inflar ou achatar artificialmente os dados.

Dessa forma, preenchemos os dados faltantes sem comprometer a qualidade da análise ou da modelagem futura.


In [ ]:
#Preencher com mediana (tempo médio tem sentido contínuo e comparável):
df["vl_medio_tempo_questionario"] = df["vl_medio_tempo_questionario"].fillna(df["vl_medio_tempo_questionario"].median())
df["vl_medio_tempo_questionario_avaliado"] = df["vl_medio_tempo_questionario_avaliado"].fillna(df["vl_medio_tempo_questionario_avaliado"].median())

In [ ]:
df.head()

### Análise de Engajamento por Cidade

Comparamos a média de interações dos alunos por cidade, permitindo identificar regiões com maior ou menor participação.

In [ ]:
# Média de engajamento por cidade
city_stats = df.groupby('ds_cidade_usuario')[['nr_interacoes_usuario',
                                              'vl_desempenho_usuario',
                                              'nr_questionarios_finalizados']].mean().reset_index()

# Gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(data=city_stats, x='ds_cidade_usuario', y='nr_interacoes_usuario')
plt.xticks(rotation=45)
plt.title('Média de Interações por Cidade')
plt.xlabel('Cidade')
plt.ylabel('Média de Interações')
plt.show()

### Distribuição do Desempenho Geral dos Alunos

O gráfico de desempenho mostra como os alunos estão performando academicamente, com base na métrica `vl_desempenho_usuario`, que varia de 0 a 1. Indicando uma concentração de alunos com ótimo desempenho e muitos com nenhum

In [ ]:
df['vl_desempenho_usuario'] = pd.to_numeric(df['vl_desempenho_usuario'], errors='coerce')
media_geral = df['vl_desempenho_usuario'].mean()

plt.figure(figsize=(10, 6))
sns.histplot(df['vl_desempenho_usuario'], bins=30, kde=True, color='skyblue')
plt.axvline(media_geral, color='red', linestyle='--', linewidth=2, label=f'Média: {media_geral:.2f}')
plt.title('Distribuição do Desempenho dos Alunos')
plt.xlabel('Desempenho')
plt.ylabel('Frequência')
plt.legend()
plt.tight_layout()
plt.show()


### Distribuição dos Dias Desde o Último Acesso

Esse gráfico analisa o comportamento de uso da plataforma, mostrando há quantos dias cada usuário realizou seu último acesso.

In [ ]:

# Selecionar  a coluna de dias desde o último acesso
dias = df['nr_dias_desde_ultimo_acesso']

# Calcular a média
media_dias = dias.mean()

# Plotar histograma com KDE (curva de densidade)
plt.figure(figsize=(10, 6))
sns.histplot(dias, bins=30, kde=True, color='skyblue')

# Adicionar linha da média
plt.axvline(media_dias, color='red', linestyle='--', linewidth=2, label=f'Média: {media_dias:.2f} dias')

# Títulos e eixos
plt.title("Distribuição dos Dias Desde o Último Acesso dos Usuários")
plt.xlabel("Dias Desde Último Acesso")
plt.ylabel("Frequência")
plt.legend()
plt.tight_layout()
plt.show()


Com base nessas análises sobre engajamento e desempenho dos alunos, criaremos uma variável chamada `evadiu`.  
Essa variável simula o comportamento de evasão, permitindo que possamos identificar alunos com maior risco de abandono do curso.

#### Criação da variável `evadiu`

Criamos uma nova coluna `evadiu` para simular o comportamento de evasão, utilizando a seguinte lógica:

> Se o aluno **nunca obteve desempenho** (`vl_desempenho_usuario = 0`) e está **há mais de 60 dias sem acessar a plataforma**, consideramos que ele evadiu.

Essa coluna é binária:
- `1` → aluno evadiu
- `0` → aluno ativo ou ainda engajado


In [ ]:
# Criar coluna 'evadiu': alunos com desempenho 0 e sem acessar há mais de 60 dias
df["vl_desempenho_usuario"] = pd.to_numeric(df["vl_desempenho_usuario"], errors="coerce")
df["evadiu"] = df["nr_dias_desde_ultimo_acesso"] <= 12

#### Comparação entre evasores e não evasores

Com a variável `evadiu` criada, comparamos os dois grupos em relação a:
- Número de interações
- Questionários finalizados
- Submissões de código
- Desempenho geral

Isso nos ajuda a entender quais comportamentos estão mais associados à evasão.

In [ ]:
# Comparar médias de variáveis entre evasores e não evasores
media_por_grupo = df.groupby("evadiu")[
    ["nr_interacoes_usuario", "nr_questionarios_finalizados", "vl_desempenho_usuario", "nr_submissoes_codigo"]
].mean().round(2)

media_por_grupo


#### Classificação de Perfis de Risco

A partir das variáveis `vl_desempenho_usuario` e `nr_dias_desde_ultimo_acesso`, classificamos cada aluno em um dos seguintes perfis:

| Perfil            | Critério                                              |
|-------------------|-------------------------------------------------------|
| Alto Risco        | Desempenho = 0 e sem acesso há mais de 60 dias        |
| Reengajamento     | Bom desempenho, mas inativo há mais de 60 dias        |
| Apoio Pedagógico  | Acessa, mas desempenho muito baixo                    |
| Estável           | Acessa com frequência e tem bom desempenho            |


In [ ]:
# Classificar alunos em perfis com base em desempenho e atividade
def perfil(row):
    if row["vl_desempenho_usuario"] == 0 and row["nr_dias_desde_ultimo_acesso"] > 60:
        return "Alto Risco"
    elif row["vl_desempenho_usuario"] > 0.7 and row["nr_dias_desde_ultimo_acesso"] > 60:
        return "Reengajamento"
    elif row["vl_desempenho_usuario"] < 0.3:
        return "Apoio Pedagógico"
    else:
        return "Estável"

df["perfil"] = df.apply(perfil, axis=1)


#### Contagem de Perfis

Finalizamos com a contagem de alunos em cada perfil. Isso nos ajuda a:

- Visualizar a distribuição de risco na base de dados
- Compreender o tamanho de cada grupo e onde priorizar intervenções

Essa segmentação é essencial tanto para análise quanto para futuras estratégias de reengajamento ou previsão automatizada da evasão.

In [ ]:
# Contar a quantidade de alunos em cada perfil
df["perfil"].value_counts().reset_index().rename(columns={"index": "perfil", "perfil": "quantidade"})


### Pré-Processamento dos Dados para Modelagem

**Seleção de Atributos**: removemos variáveis que não serão usadas como entrada no modelo, como:
   - `evadiu` (alvo da predição)
   - `perfil` (já é um agrupamento explicativo derivado das features)
   - `ds_cidade_usuario` (categórica não transformada)

In [ ]:
y = df["evadiu"]

In [ ]:
X = df.drop(columns=["evadiu", "perfil", "ds_cidade_usuario", "nr_dias_desde_ultimo_acesso"], errors="ignore")

**Exclusão de colunas do tipo `datetime64`:**
   - Modelos de ML não entendem datas diretamente.
   - Em vez disso, já transformamos essas colunas em atributos úteis como `nr_dias_desde_ultimo_acesso`.

In [ ]:
X = X.select_dtypes(exclude=["datetime64[ns]"])

Usamos `StandardScaler` para padronizar as variáveis numéricas, transformando-as para que tenham **média 0 e desvio padrão 1**. Isso é especialmente importante para algoritmos como redes neurais e modelos baseados em distância (ex: SVM), que são sensíveis à escala dos dados.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

**Separação dos Conjuntos de Treino e Teste**: dividimos os dados em 80% para treino e 20% para teste, garantindo que a proporção de evasores e não evasores seja mantida (com `stratify`).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

**Verificação de Balanceamento**: imprimimos a distribuição das classes (evadiu = 0 ou 1) no conjunto de treino, o que ajuda a identificar se técnicas adicionais de balanceamento serão necessárias no modelo.

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print("Distribuição de 'evadiu' no treino:", dict(zip(unique, counts)))

## Modelagem

Importação de bibliotecas

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold


Usamos regressão logística por ser um modelo simples e interpretável, ótimo para começar problemas de classificação binária como “evadiu ou não”.

In [ ]:
## Regressão Logística
log_model = LogisticRegression(random_state=42, max_iter=1000)
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)

print("Relatório - Regressão Logística")
print(classification_report(y_test, y_pred_log))

Neste teste, o modelo teve 100% de acurácia e f1-score, acertando todos os 41 casos (38 não evadiram, 3 evadiram). Isso indica um ótimo desempenho, mas como os dados são poucos, **é importante confirmar essa performance com validação cruzada.**


**Interpretação do Modelo de Regressão Logística**

- **feature_names = X_train.columns**  
  Pega os nomes das variáveis (features) do dataset de treino.

- **coefficients = log_model.coef_[0]**  
  Obtém os coeficientes (pesos) aprendidos pelo modelo para cada variável.

- Em seguida, criamos um DataFrame chamado **coef_df** que reúne:  
  - A feature (variável)  
  - Seu coeficiente  
  - O impacto em termos de **odds ratio**, calculado por `np.exp(coeficiente)`

- Ordenamos esse DataFrame em ordem decrescente pelo valor dos coeficientes para identificar as features que mais impactam o modelo.

- Por fim, imprimimos essa tabela para entender quais variáveis **aumentam** ou **diminuem** a chance do evento previsto (exemplo: evasão).


In [ ]:
feature_names = X_train.columns
coefficients = log_model.coef_[0]

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coeficiente': coefficients,
    'Impacto': np.exp(coefficients) 
}).sort_values(by='Coeficiente', ascending=False)

print(coef_df)


Usamos Random Forest porque é um modelo robusto, capaz de lidar com dados não lineares e identificar relações complexas entre as variáveis.

In [ ]:
## Random Forest
rf_model = RandomForestClassifier(
    random_state=42,
    max_depth=5,
    min_samples_leaf=2,
    n_estimators=700
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\nRelatório - Random Forest")
print(classification_report(y_test, y_pred_rf))

O modelo teve alta precisão geral, mas deixou passar 1 aluno evadido (recall = 0.67 para a classe 1). Isso indica que ele foi mais conservador nas previsões de evasão, preferindo errar por “excesso de cautela”.

**Interpretação Global com SHAP para Random Forest**

- **explainer_rf = shap.TreeExplainer(rf_model)**  
  Cria um objeto explainer que entende o funcionamento interno do modelo Random Forest.

- **shap_values_rf = explainer_rf.shap_values(X_test)**  
  Calcula os valores SHAP para cada feature em cada amostra do conjunto de teste.  
  Esses valores medem o impacto de cada variável na decisão do modelo para cada observação.

- **shap.summary_plot(shap_values_rf[1], X_test, feature_names=X_test.columns)**  
  Gera um gráfico resumo que mostra:  
  - Quais variáveis tiveram mais influência na predição do modelo em todo o conjunto de teste.  
  - A direção do impacto (cores indicam valores altos ou baixos da feature).  


In [ ]:
import shap

explainer_rf = shap.TreeExplainer(rf_model)
shap_values_rf = explainer_rf.shap_values(X_test)

shap.summary_plot(shap_values_rf[1], X_test, feature_names=X_test.columns)


Aqui comparamos graficamente os acertos e erros de cada modelo (Logistic Regression e Random Forest). As matrizes mostram quantos alunos foram corretamente ou incorretamente classificados como evadidos ou não evadidos.

In [ ]:
# Matrizes de confusão
conf_matrix_log = confusion_matrix(y_test, y_pred_log)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)

# Visualização das matrizes de confusão
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix_log, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de Confusão - Logistic Regression")
plt.xlabel("Previsto")
plt.ylabel("Real")

plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_rf, annot=True, fmt="d", cmap="Greens")
plt.title("Matriz de Confusão - Random Forest")
plt.xlabel("Previsto")
plt.ylabel("Real")

plt.tight_layout()
plt.show()

Utilizamos validação cruzada com 5 divisões (stratified) para avaliar a estabilidade dos modelos. O F1-score médio e o desvio padrão mostram se o desempenho se mantém consistente em diferentes partes do conjunto de treino.

In [ ]:
# Validação Cruzada para conferir estabilidade dos modelos
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_log = cross_val_score(log_model, X_train, y_train, cv=cv, scoring="f1")
scores_rf = cross_val_score(rf_model, X_train, y_train, cv=cv, scoring="f1")

print("\nF1-score - Cross Validation")
print(f"Regressão Logística: {scores_log.mean():.4f} ± {scores_log.std():.4f}")
print(f"Random Forest: {scores_rf.mean():.4f} ± {scores_rf.std():.4f}")

Regressão Logística:
Obteve um F1-score médio de 0.8933 ± 0.1373, indicando bom desempenho, mas com uma leve variação entre os folds (±13%).

Random Forest:
Teve F1-score médio de 0.9333 ± 0.1333, ou seja, desempenho um pouco melhor e com estabilidade semelhante.

Ambos os modelos mostraram consistência, mas o Random Forest teve um desempenho geral ligeiramente superior na média. Isso reforça sua robustez para o problema, mesmo com um conjunto de dados pequeno.

#### Rede Neural

importação de bibliotecas

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Criar o modelo sequencial
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # saída binária

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping para evitar overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Treinar o modelo
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)



In [ ]:
# Avaliar no conjunto de teste
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nRede Neural - Accuracy no teste: {accuracy:.4f}")

Aqui verificamos se a rede neural acertou de fato os alunos evadidos (classe 1), e não apenas os não evadidos. Mesmo com accuracy 100%, a matriz de confusão confirma se houve falsos negativos (evasores ignorados), o que é crítico no nosso problema.


In [ ]:
from sklearn.metrics import confusion_matrix

# Previsão
y_pred_nn = model.predict(X_test)
y_pred_nn_classes = (y_pred_nn > 0.5).astype("int")

# Matriz de confusão
conf_matrix_nn = confusion_matrix(y_test, y_pred_nn_classes)

# Visualização
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix_nn, annot=True, fmt="d", cmap="Purples")
plt.title("Matriz de Confusão - Rede Neural")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.show()


O modelo previu corretamente todos os casos, incluindo os 3 alunos evadidos (classe 1).
Isso confirma que a rede neural não apenas teve alta acurácia, mas também alto recall, que é crucial neste problema, pois queremos identificar quem está em risco de evasão.

Como não houve falsos negativos nem positivos, o modelo parece estar bem treinado e generalizando bem neste conjunto de teste.


In [ ]:
import matplotlib.pyplot as plt

# Plotando a acurácia e perda durante o treinamento
plt.plot(history.history['accuracy'], label='Acurácia - Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia - Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.title('Evolução da Acurácia')
plt.show()

Usamos o SHAP DeepExplainer para explicar o impacto das variáveis na predição da rede neural. Selecionamos uma amostra do treino como referência, calculamos os valores SHAP no teste e geramos um gráfico resumo que mostra a importância e direção das features no modelo. Também criamos uma explicação local para o primeiro exemplo, detalhando como cada variável influenciou aquela predição específica.

In [ ]:

X_background = X_train.sample(100, random_state=42) 

explainer = shap.DeepExplainer(model, X_background)

shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test, feature_names=X_test.columns)

shap.force_plot(explainer.expected_value[0], shap_values[0][0], X_test.iloc[0], matplotlib=True)


## Interpretação (XAI)

### Importância do XAI (Explainable AI)

Em problemas críticos como a predição de evasão escolar, não basta apenas obter um modelo com alta acurácia. É fundamental entender por que o modelo toma determinadas decisões, especialmente em cenários que impactam vidas e demandam ações assertivas. A Explainable AI (XAI) surge para dar transparência aos modelos de machine learning, permitindo que desenvolvedores, analistas e stakeholders compreendam quais variáveis influenciam mais as predições e em que sentido.
Essa interpretação ajuda a evitar vieses, identificar erros de modelagem e aumentar a confiança no sistema automatizado. Além disso, quando as decisões são explicáveis, fica mais fácil validar o modelo com conhecimento de domínio e garantir que ele esteja alinhado com as expectativas do problema.

### Por que escolhemos o SHAP?

O **SHAP** (SHapley Additive exPlanations) é uma das técnicas mais robustas e teoricamente fundamentadas para explicar modelos complexos como redes neurais profundas. Atribui de forma justa a contribuição de cada feature para a predição, considerando todas as combinações possíveis.

Algumas vantagens do SHAP:
- Consistência e precisão nas explicações.
- Explicações tanto globais (importância geral das features no modelo) quanto locais (por exemplo, o motivo de uma predição individual).
- Suporte para diversos tipos de modelos, incluindo redes neurais via DeepExplainer.
- Visualizações intuitivas e interpretáveis que facilitam a comunicação dos resultados.

### Avaliação final no conjunto de teste real
Usamos o dataset oficial de teste para validar o modelo final como se estivéssemos em produção.

In [ ]:
df_teste_real = pd.read_csv("dados_projeto_evasao_teste - Copia.csv")